In [1]:
pwd

'/home/wsuser/work'

In [2]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='CSHhszw9ddSX4KbBSVPt8j_86ntcqBN_oXE9xbVB0Rbe',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'vehicledamageprediction-donotdelete-pr-izzfghakpkjpew'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
ls

Dataset/


In [7]:
imageSize = [224, 224]

trainPath = r"/home/wsuser/work/Dataset/body/training"

testPath = r"/home/wsuser/work/Dataset/body/validation"

In [8]:
vgg = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [9]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [10]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [11]:
prediction = Dense(3, activation='softmax')(x)

In [12]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [13]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [15]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [16]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [17]:
cd

/home/wsuser


In [18]:
import sys
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=979//10,
  validation_steps=171//10)

/tmp/wsuser/ipykernel_164/2568811991.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/25
97/97 [==============================] - 344s 4s/step - loss: 1.1917 - accuracy: 0.5408 - val_loss: 1.3177 - val_accuracy: 0.6000
Epoch 2/25
97/97 [==============================] - 344s 4s/step - loss: 0.7385 - accuracy: 0.7265 - val_loss: 0.9082 - val_accuracy: 0.6824
Epoch 3/25
97/97 [==============================] - 349s 4s/step - loss: 0.6298 - accuracy: 0.7647 - val_loss: 1.0590 - val_accuracy: 0.6412
Epoch 4/25
97/97 [==============================] - 346s 4s/step - loss: 0.5005 - accuracy: 0.8266 - val_loss: 0.9600 - val_accuracy: 0.6471
Epoch 5/25
97/97 [==============================] - 342s 4s/step - loss: 0.3400 - accuracy: 0.8741 - val_loss: 1.0763 - val_accuracy: 0.6529
Epoch 6/25
97/97 [==============================] - 344s 4s/step - loss: 0.3208 - accuracy: 0.8679 - val_loss: 0.9483 - val_accuracy: 0.7000
Epoch 7/25
97/97 [==============================] - 343s 4s/step - loss: 0.2679 - accuracy: 0.8978 - val_loss: 0.9654 - val_accuracy: 0.7000
Epoch 8/25
97

In [19]:
#save the model
model.save('body.h5')

In [20]:
!tar -zcvf body.tgz body.h5

body.h5


In [21]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 10.9 MB/s eta 0:00:01


In [22]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey": "d0gUkEa-VGHV4xv97S6mdz03ov8YrlOM55AOvGr79yb6"
                  }
client = APIClient(wml_credentials)

In [23]:
client = APIClient(wml_credentials)

In [25]:
def guid_from_space_name(client, Damage_predictor):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] ==Damage_predictor)['metadata']['id'])

In [26]:
space_uid = guid_from_space_name(client, 'Damage_predictor')
print("SPACE UID = " + space_uid )

SPACE UID = 72375dcb-d3c1-4acc-82db-8020e959bcd7


In [27]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [29]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [30]:
ls

body.h5  body.tgz  jars/  work/


In [31]:
import tensorflow
tensorflow.__version__

'2.7.2'

In [32]:
model_details=client.repository.store_model(model='body.tgz',
            meta_props={
                        client.repository.ModelMetaNames.NAME:"CNN",
                        client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',  
                        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

In [33]:
model_id=client.repository.get_model_id(model_details)

In [34]:
model_id

'8c297ea4-a2b6-483b-8927-b0bb7024220e'

In [35]:
client.repository.download('8c297ea4-a2b6-483b-8927-b0bb7024220e','bodycnn.tgz')

Successfully saved model content to file: 'bodycnn.tgz'


'/home/wsuser/bodycnn.tgz'